## **Import the Libaries**

In [8]:
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
import tensorflow_hub as hub

import os
import random
import shutil
from math import ceil, sqrt
import matplotlib.pyplot as plt

## **Meander**

In [9]:
os.listdir('/content/drive/MyDrive/Dataset/Meander/Train')

['Patient', 'Healthy']

In [10]:
os.listdir('/content/drive/MyDrive/Dataset/Meander/Test')

['Healthy', 'Patient']

In [11]:
TRAIN_DIR='/content/drive/MyDrive/Dataset/Meander/Train'

In [12]:
train_dir='/content/drive/MyDrive/Dataset/Meander/Train'
test_dir='/content/drive/MyDrive/Dataset/Meander/Test'

In [13]:
# Create the training and validation directories if they don't exist
train_dir = 'train'
valid_dir = 'val'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

for subdir in os.listdir(TRAIN_DIR):
    subdir_path = os.path.join(TRAIN_DIR, subdir)

    # Create corresponding subdirectories in train and val directories
    train_subdir = os.path.join(train_dir, subdir)
    valid_subdir = os.path.join(valid_dir, subdir)

    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(valid_subdir, exist_ok=True)

    all_images = os.listdir(subdir_path)

    num_validation = int(len(all_images) * 0.2)
    validation_images = random.sample(all_images, num_validation)

    for img in all_images:
        source_path = os.path.join(subdir_path, img)
        dest_dir = valid_subdir if img in validation_images else train_subdir
        dest_path = os.path.join(dest_dir, img)
        shutil.copy(source_path, dest_path)

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,      # Rescale pixel values to [0, 1]
    rotation_range=25,      # Randomly rotate images by up to 25 degrees
    width_shift_range=0.3,  # Randomly shift the width of images
    height_shift_range=0.3, # Randomly shift the height of images
    horizontal_flip=True,   # Randomly flip images horizontally
    shear_range=0.3,        # Apply shear transformations
    zoom_range=0.4,         # Randomly zoom into images
    fill_mode='nearest'     # Fill empty pixels with the nearest value
)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)
# Create data generators for training and validation data
batch_size = 32
image_size = (150, 150)

train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = test_datagen.flow_from_directory(
    '/content/val',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)




Found 245 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [15]:
efficient_net = hub.KerasLayer("https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet21k-b3-feature-vector/versions/1",trainable=False)

In [16]:

model_meander = keras.Sequential([
    efficient_net,
    keras.layers.Flatten(),  # Add a Flatten layer to reshape the output
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(6, activation='softmax'),
    keras.layers.Dense(1, activation='sigmoid'),
])

model_meander.build((None, image_size[0], image_size[1], 3))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(5e-4, decay_steps=10000, decay_rate=0.9)
model_meander.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping( monitor='val_accuracy',patience=5,restore_best_weights=True)
model_meander.save("image_model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
history = model_meander.fit(train_generator, epochs=100, validation_data=val_generator)


# Save the model
model_meander.save("meander_model.h5")

# Optionally, save the training history as well
import pickle

with open('training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)


Epoch 1/100
8/8 [==============================] - 32s 2s/step - loss: 0.6429 - accuracy: 0.6449 - val_loss: 0.6560 - val_accuracy: 0.7167
Epoch 2/100
8/8 [==============================] - 15s 2s/step - loss: 0.6036 - accuracy: 0.7306 - val_loss: 0.6314 - val_accuracy: 0.7000
Epoch 3/100
8/8 [==============================] - 17s 2s/step - loss: 0.5953 - accuracy: 0.7388 - val_loss: 0.6190 - val_accuracy: 0.7000
Epoch 4/100
8/8 [==============================] - 17s 2s/step - loss: 0.5728 - accuracy: 0.7592 - val_loss: 0.6074 - val_accuracy: 0.7333
Epoch 5/100
8/8 [==============================] - 15s 2s/step - loss: 0.5607 - accuracy: 0.8245 - val_loss: 0.6043 - val_accuracy: 0.7667
Epoch 6/100
8/8 [==============================] - 15s 2s/step - loss: 0.5537 - accuracy: 0.7959 - val_loss: 0.5975 - val_accuracy: 0.7333
Epoch 7/100
8/8 [==============================] - 17s 2s/step - loss: 0.5559 - accuracy: 0.7837 - val_loss: 0.5596 - val_accuracy: 0.8500
Epoch 8/100
8/8 [==========

In [18]:
image_prediction = model_meander.predict(val_generator)
loss, accuracy = model_meander.evaluate(val_generator)
print(f'Test loss: {loss:.4f}, accuracy: {accuracy:.4f}')

2/2 [==============================] - 2s 1s/step - loss: 0.3878 - accuracy: 0.9167
Test loss: 0.3878, accuracy: 0.9167


## **Spiral**

In [19]:
os.listdir('/content/drive/MyDrive/Dataset/Spiral/Train')

['Patient', 'Healthy']

In [20]:
os.listdir('/content/drive/MyDrive/Dataset/Spiral/Test')

['Patient', 'Healthy']

In [21]:
TRAIN_DIR='/content/drive/MyDrive/Dataset/Spiral/Train'

In [22]:
train_dir='/content/drive/MyDrive/Dataset/Spiral/Train'
test_dir='/content/drive/MyDrive/Dataset/Spiral/Test'

In [23]:
# Create the training and validation directories if they don't exist
train_dir = 'train1'
valid_dir = 'val1'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

for subdir in os.listdir(TRAIN_DIR):
    subdir_path = os.path.join(TRAIN_DIR, subdir)

    # Create corresponding subdirectories in train and val directories
    train_subdir = os.path.join(train_dir, subdir)
    valid_subdir = os.path.join(valid_dir, subdir)

    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(valid_subdir, exist_ok=True)

    all_images = os.listdir(subdir_path)

    num_validation = int(len(all_images) * 0.2)
    validation_images = random.sample(all_images, num_validation)

    for img in all_images:
        source_path = os.path.join(subdir_path, img)
        dest_dir = valid_subdir if img in validation_images else train_subdir
        dest_path = os.path.join(dest_dir, img)
        shutil.copy(source_path, dest_path)

In [24]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,      # Rescale pixel values to [0, 1]
    rotation_range=25,      # Randomly rotate images by up to 25 degrees
    width_shift_range=0.3,  # Randomly shift the width of images
    height_shift_range=0.3, # Randomly shift the height of images
    horizontal_flip=True,   # Randomly flip images horizontally
    shear_range=0.3,        # Apply shear transformations
    zoom_range=0.4,         # Randomly zoom into images
    fill_mode='nearest'     # Fill empty pixels with the nearest value
)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)
# Create data generators for training and validation data
batch_size = 32
image_size = (150, 150)

train_generator = train_datagen.flow_from_directory(
    '/content/train1',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = test_datagen.flow_from_directory(
    '/content/val1',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)




Found 280 images belonging to 2 classes.
Found 69 images belonging to 2 classes.


In [25]:
efficient_net = hub.KerasLayer("https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet21k-b3-feature-vector/versions/1",trainable=False)

In [26]:

model_spiral = keras.Sequential([
    efficient_net,
    keras.layers.Flatten(),  # Add a Flatten layer to reshape the output
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(6, activation='softmax'),
    keras.layers.Dense(1, activation='sigmoid'),
])

model_spiral.build((None, image_size[0], image_size[1], 3))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(5e-4, decay_steps=10000, decay_rate=0.9)
model_spiral.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping( monitor='val_accuracy',patience=5,restore_best_weights=True)

In [27]:
history = model_spiral.fit(train_generator, epochs=100, validation_data=val_generator)


# Save the model
model_spiral.save("spiral_model.h5")

# Optionally, save the training history as well
import pickle

with open('training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)


Epoch 1/100
9/9 [==============================] - 33s 2s/step - loss: 0.6584 - accuracy: 0.6179 - val_loss: 0.6821 - val_accuracy: 0.4928
Epoch 2/100
9/9 [==============================] - 17s 2s/step - loss: 0.6336 - accuracy: 0.6500 - val_loss: 0.6506 - val_accuracy: 0.6522
Epoch 3/100
9/9 [==============================] - 18s 2s/step - loss: 0.6156 - accuracy: 0.6964 - val_loss: 0.6209 - val_accuracy: 0.6667
Epoch 4/100
9/9 [==============================] - 16s 2s/step - loss: 0.5924 - accuracy: 0.7500 - val_loss: 0.6076 - val_accuracy: 0.7101
Epoch 5/100
9/9 [==============================] - 19s 2s/step - loss: 0.5770 - accuracy: 0.7929 - val_loss: 0.5900 - val_accuracy: 0.7536
Epoch 6/100
9/9 [==============================] - 17s 2s/step - loss: 0.5925 - accuracy: 0.7179 - val_loss: 0.5802 - val_accuracy: 0.7391
Epoch 7/100
9/9 [==============================] - 17s 2s/step - loss: 0.5752 - accuracy: 0.7786 - val_loss: 0.5700 - val_accuracy: 0.7391
Epoch 8/100
9/9 [==========

In [28]:
image_predictions1 = model_spiral.predict(val_generator)
loss1, accuracy1 = model_spiral.evaluate(val_generator)
print(f'Test loss: {loss1:.4f}, accuracy: {accuracy1:.4f}')

3/3 [==============================] - 3s 724ms/step - loss: 0.5460 - accuracy: 0.7246
Test loss: 0.5460, accuracy: 0.7246


## **CatBoost**

In [29]:
pip install catboost

In [30]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("/content/drive/MyDrive/ParkinsonDisease/MDVR_all_features_chunks.csv")

# Import LabelEncoder from sklearn
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode the first column of the DataFrame
df['voiceID'] = label_encoder.fit_transform(df['voiceID'])

# Save the modified DataFrame back to a CSV file
df.to_csv("labeled.csv", index=False)


In [31]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the dataset from a CSV file
input_file = 'labeled.csv'  # Replace with the path to your CSV file
output_file = 'normalized.csv'  # Replace with the desired output path

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv(input_file)

# Select numeric columns (excluding the first and last columns) for normalization
numeric_columns = data.select_dtypes(include=['number']).columns[1:-1]  # Exclude the first and last columns

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Normalize the selected columns
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Save the normalized data to a new CSV file
data.to_csv(output_file, index=False)

print("Normalization complete. Normalized data saved to", output_file)


Normalization complete. Normalized data saved to normalized.csv


In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Load the dataset
data = pd.read_csv("normalized.csv")

# Split features (X) and labels (y)
X = data.iloc[:, 1:-1]  # Exclude the first and last columns
y = data['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [33]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import pandas as pd
from sklearn.metrics import r2_score

# Load the dataset
data = pd.read_csv("normalized.csv")

# Split features (X) and labels (y)
X = data.iloc[:, 1:-1]  # Exclude the first and last columns
y = data['label']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define CatBoostClassifier
model_voice = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           loss_function='Logloss',
                           eval_metric='Accuracy',
                           random_seed=42,
                           logging_level='Silent')

# Train the model
model_voice.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

# Save the model
model_voice.save_model("catboost_model")

print("Model saved successfully.")


# Make predictions on test set
y_pred = model_voice.predict(X_test)

voice_prediction = y_pred

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calculate precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

from sklearn.metrics import r2_score

# Assuming you have y_true and y_pred variables containing ground truth and predicted values respectively
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)


# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Model saved successfully.
Accuracy: 0.9239766081871345
Recall: 0.9090909090909091
Precision: 0.8955223880597015
F1 Score: 0.9022556390977443
R2 Score: 0.6792207792207792
Confusion Matrix:
[[98  7]
 [ 6 60]]


## **FUSED MODEL**

In [35]:
from tensorflow.keras.models import load_model
import tensorflow_hub as hub
from catboost import CatBoostClassifier
# Load the trained models
model_spiral = load_model('spiral_model.h5', custom_objects={'KerasLayer': hub.KerasLayer})
model_meander = load_model('meander_model.h5', custom_objects={'KerasLayer': hub.KerasLayer})
model_voice = CatBoostClassifier().load_model("catboost_model")


In [36]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your test directories
test_dir_spiral = '/content/drive/MyDrive/Dataset/Spiral/Val'
test_dir_meander = '/content/drive/MyDrive/Dataset/Meander/Val'  # replace with your meander test directory

# Initialize an ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Create data generators for test data
batch_size = 32
image_size = (150, 150)

test_generator_spiral = test_datagen.flow_from_directory(
    test_dir_spiral,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator_meander = test_datagen.flow_from_directory(
    test_dir_meander,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)


# Get the actual labels from your test data
labels_spiral = test_generator_spiral.classes
labels_meander = test_generator_meander.classes



Found 54 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


In [38]:
import pandas as pd

# Load your dataset
df = pd.read_csv('test.csv')  # replace with the path to your csv file

# Separate the features and labels
features = df.iloc[:, 1:-1]  # all rows, all columns except the first (ID) and the last (label)
labels = df.iloc[:, -1]  # all rows, last column (label)


In [39]:
# Assume that you have three trained models: image_model1, image_model2, and voice_model
# And you have three test datasets: test_data1, test_data2, and test_data_voice

# Get the predictions from all models
# Now you can get the predictions from both models
preds_spiral = model_spiral.predict(test_generator_spiral)
preds_meander = model_meander.predict(test_generator_meander)
# Get the predictions from the model
preds_voice = model_voice.predict(features)

print(preds_spiral)
print(preds_meander)
print(preds_voice)



2/2 [==============================] - 8s 5s/step
[[0.34727523]
 [0.34727523]
 [0.34727708]
 [0.7930116 ]
 [0.3475517 ]
 [0.79325897]
 [0.7933623 ]
 [0.34728268]
 [0.7934183 ]
 [0.79344004]
 [0.34731352]
 [0.7933796 ]
 [0.7933142 ]
 [0.7932185 ]
 [0.7933214 ]
 [0.3472758 ]
 [0.34736085]
 [0.4813201 ]
 [0.7929347 ]
 [0.34727523]
 [0.34727532]
 [0.36847383]
 [0.78506726]
 [0.34729657]
 [0.34727523]
 [0.352749  ]
 [0.7933525 ]
 [0.34727687]
 [0.34727523]
 [0.36451706]
 [0.3472753 ]
 [0.55818796]
 [0.34727523]
 [0.792312  ]
 [0.34727797]
 [0.34727612]
 [0.79159003]
 [0.34728146]
 [0.34727523]
 [0.34727535]
 [0.4036928 ]
 [0.78560233]
 [0.34727523]
 [0.34727523]
 [0.34728092]
 [0.34727523]
 [0.7934001 ]
 [0.46101922]
 [0.3472752 ]
 [0.793409  ]
 [0.78971887]
 [0.6840871 ]
 [0.34727544]
 [0.34727615]]
[[0.7963106 ]
 [0.7965232 ]
 [0.79652005]
 [0.79650676]
 [0.79245037]
 [0.38732442]
 [0.38440087]
 [0.38426316]
 [0.38424832]
 [0.7941699 ]
 [0.7601424 ]
 [0.7964322 ]
 [0.4338049 ]
 [0.7964362

In [40]:
# Round the predictions to the nearest integer
preds_image1_rounded = np.round(preds_spiral)
preds_image2_rounded = np.round(preds_meander)
# Round the predictions to the nearest integer
preds_voice_rounded = np.round(preds_voice)



In [41]:
# Stack the rounded predictions
stacked_preds_rounded = np.column_stack((preds_image1_rounded, preds_image2_rounded, preds_voice_rounded))


In [43]:
from scipy import stats


In [53]:
# Use the mode of the rounded predictions as the final prediction
final_preds, _ = stats.mode(stacked_preds_rounded, axis=1)
final_preds = final_preds.ravel()  # flatten the array


In [61]:
from sklearn.metrics import accuracy_score

# Get the actual labels from your test data
labels_spiral = test_generator_spiral.classes
labels_meander = test_generator_meander.classes
final_test = labels
# Calculate the accuracy of the fused model
accuracy = accuracy_score(final_test, final_preds)

print(f"Accuracy of the fused model: {accuracy * 100:.2f}%")

Accuracy of the fused model: 93.22%
